In [ ]:
!pip install https://github.com/UCREL/pymusas-models/releases/download/en_dual_none_contextual-0.3.1/en_dual_none_contextual-0.3.1-py3-none-any.whl
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 917 kB 23.7 MB/s 
     |████████████████████████████████| 51 kB 707 kB/s 
2022-10-27 19:14:27.879710: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 26.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive

/content/drive


In [ ]:
cd MyDrive

/content/drive/MyDrive


In [ ]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint



#df = pd.read_csv('Datasets/discord/writingprompts_discord_hate.csv')
df = pd.read_csv('Datasets/Reddit/sports_subreddit_hate.csv',lineterminator='\n',on_bad_lines='skip')
df=df.loc[df['hate'] == 1.0]
df=df.sample(10000)
df=df.dropna()

# data=df.content

# # # Remove Emails
# data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# # # Remove new line characters
# # data = [re.sub('\s+', ' ', sent) for sent in data]

# # # Remove distracting single quotes
# # data = [re.sub("\'", "", sent) for sent in data]

# #pprint(data[2:])
# df=data
df

,Unnamed: 0,comment_text,author,timestamp,hate
125669,125669,Missing a panenka looks way more stupid though...,Dickinmymouth1,2022-04-27 11:39:04,1.0
159128,159128,Uploaded by weakest man ever lol,Bigfsi,2022-04-04 07:34:52,1.0
183199,183199,I sued my neighbor for backing his car into my...,tafor83,2022-03-19 03:33:02,1.0
180791,180791,Hey it was a slap! Juwan really showing off th...,scarystuffdoc,2022-03-20 14:48:32,1.0
34552,34552,Ahh. So anti-pig cruelty PETA protestor.,WintersTablet,2022-07-05 12:01:31,1.0
...,...,...,...,...,...
62537,62537,"""WTF?!?"" - Ryne Sandberg",NoSweatBetting,2022-06-16 23:26:42,1.0
32331,32331,It’s not protesting the government she’s tryin...,youpeopleareannoying,2022-07-07 17:07:48,1.0
84867,84867,"Just remember, Voting republican is also about...",Uumm_wat,2022-05-28 20:59:20,1.0
224248,224248,They tend to do that when they are loaded up w...,jn-indianwood,2022-02-21 23:04:01,1.0


In [ ]:
import numpy as np
import multiprocessing as mp

import string
import spacy 
from sklearn.base import TransformerMixin, BaseEstimator


nlp = spacy.load("en_core_web_sm")

class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self,
                 nlp = nlp,
                 n_jobs=1):
        """
        Text preprocessing transformer includes steps:
            1. Punctuation removal
            2. Stop words removal
            3. Lemmatization

        nlp  - spacy model
        n_jobs - parallel jobs to run
        """
        self.nlp = nlp
        self.n_jobs = n_jobs

    def fit(self, X, y=None):
        return self

    def transform(self, X, *_):
        X_copy = X.copy()

        partitions = 1
        cores = mp.cpu_count()
        if self.n_jobs <= -1:
            partitions = cores
        elif self.n_jobs <= 0:
            return X_copy.apply(self._preprocess_text)
        else:
            partitions = min(self.n_jobs, cores)

        data_split = np.array_split(X_copy, partitions)
        pool = mp.Pool(cores)
        data = pd.concat(pool.map(self._preprocess_part, data_split))
        pool.close()
        pool.join()

        return data

    def _preprocess_part(self, part):
        return part.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        doc = self.nlp(text)
        removed_punct = self._remove_punct(doc)
        removed_stop_words = self._remove_stop_words(removed_punct)
        return self._lemmatize(removed_stop_words)

    def _remove_punct(self, doc):
        return (t for t in doc if t.text not in string.punctuation)

    def _remove_stop_words(self, doc):
        return (t for t in doc if not t.is_stop)

    def _lemmatize(self, doc):
        return ' '.join(t.lemma_ for t in doc)

In [ ]:
#text = TextPreprocessor(n_jobs=-1).transform(df['content'])
text = TextPreprocessor(n_jobs=-1).transform(df['comment_text'])
text=text.str.cat()
text


# output_doc = nlp(text)

# print(f'Text\tLemma\tPOS\tUSAS Tags')
# for token in output_doc:
#     print(f'{token.text}\t{token.lemma_}\t{token.pos_}\t{token._.pymusas_tags}')

'miss panenka look way stupid absolutely mockupload weak man lolsue neighbor back car house drunk instead have arrest \n\n binary train thinking fuck juvenilehey slap Juwan show duality man yearAhh anti pig cruelty PETA protestorfood pussyPeters plead guilty pay 700 fine drive illegal muffler lol \n\n find comparable sexually assault harass 22 womantest blood hghdisgrace game manright dude bigot ...  actually retardedrackets f ING MOUTH -serena ref probablydefinitely go arm trap arm armpit rip armbarhigh school girlfriend go nuts night break like say face " " \n\n ignore call text day afterward show park outside house day like school week lol outside talk tell ball break person \n\n thing bug lol gunna act like face know break8 year old drugsure country uphold law sad day justicebullshit \n\n Israel completely evacuate West Bank 2005 leave Palestine occupy \n\n Palestine respond set rocket site fire hundred rocket Israel \n\n defend country openly call extermination Jews \n\n ignorant 

In [ ]:
import spacy
import en_dual_none_contextual
import csv
from pathlib import Path
from typing import List
import ast

# We exclude the following components as we do not need them. 
nlp = spacy.load('en_core_web_sm', exclude=['parser', 'ner'])
# Load the English PyMUSAS rule based tagger in a separate spaCy pipeline
english_tagger_pipeline = en_dual_none_contextual.load()
# Adds the English PyMUSAS rule based tagger to the main spaCy pipeline
nlp.add_pipe('pymusas_rule_based_tagger', source=english_tagger_pipeline)


output_tsv_file = Path('Datasets/Results/Reddit_semantic/sports_subreddit_semantics.tsv').resolve()


with output_tsv_file.open('w', encoding='utf-8', newline='') as output_fp:
    field_names = ['Token', 'Lemma', 'UPOS', 'USAS Tags']
    tsv_writer = csv.DictWriter(output_fp, fieldnames=field_names, delimiter='\t')
    tsv_writer.writeheader()

    output_doc = nlp(text)

    for token in output_doc:
                tsv_writer.writerow({'Token': token.text,
                                     'Lemma': token.lemma_,
                                     'UPOS': token.pos_,
                                     'USAS Tags': token._.pymusas_tags})

/usr/local/lib/python3.7/dist-packages/pymusas/spacy_api/utils.py:38: UserWarning: Overwritten the spaCy Token extension `pymusas_tags` which currently has the following (default, method, getter, setter):`(None, None, None, None)`. And replacing it with the following:`(None, None, None, None)`. This would only become a problem if the the two Tuples of four are different, if they are the same there is no problem.
  warnings.warn(message)
/usr/local/lib/python3.7/dist-packages/pymusas/spacy_api/utils.py:38: UserWarning: Overwritten the spaCy Token extension `pymusas_mwe_indexes` which currently has the following (default, method, getter, setter):`(None, None, None, None)`. And replacing it with the following:`(None, None, None, None)`. This would only become a problem if the the two Tuples of four are different, if they are the same there is no problem.
  warnings.warn(message)


In [ ]:
cd Datasets

/content/drive/MyDrive/Datasets


In [ ]:
cd Results

/content/drive/MyDrive/Datasets/Results


In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv('Reddit_semantic/sports_subreddit_semantics.tsv', sep='\t')
df['USAS Tags'] = df['USAS Tags'].str.replace(r'[', '')
df['USAS Tags'] = df['USAS Tags'].str.replace(r']', '')
df['USAS Tags'] = df['USAS Tags'].str.replace(r"'", "")

i, c = np.unique(df['USAS Tags'], return_counts = True)
out = pd.Series(c, index = i)
out.to_csv('Reddit_semantic/Reddit_semantic_count/sports_subreddit_count.csv')
out

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """


A1.1.1                    89
A1.1.1, A1.1.2             8
A1.1.1, A1.4               1
A1.1.1, A1.7+, E3-@        1
A1.1.1, A1.7+, K5.1        2
                       ...  
Z8/Z6cmf                   1
Z8mf                      38
Z8mfc                     28
Z8mfn                      2
Z99                    15084
Length: 4228, dtype: int64

In [ ]:
# import pandas as pd
# import numpy as np
# df=pd.read_csv('Discord_semantic/Discord_semantic_count/Games_discord_count.csv')
# df

In [ ]:
import pandas as pd


df0=pd.read_csv('Reddit_semantic/Reddit_semantic_count/sports_subreddit_count.csv')

df=df0['Unnamed: 0'].str[:6]
df0=df0['0']
df=pd.concat([df,df0], axis=1)

df2=df.groupby("Unnamed: 0")["0"].sum().sort_values(ascending=False)
df2.to_csv('Reddit_semantic/Reddit_semantic_count/sports_subreddit_count1.csv')
#df=df.unique()
df2

Unnamed: 0
Z99       15084
T1.3      10772
Z1mf,      8917
Z2         8276
PUNCT      2255
          ...  
G1.2/Q        1
G2.1/A        1
G2.2+/        1
G3, A1        1
S7.4+/        1
Name: 0, Length: 1944, dtype: int64